# Setup of New Database

You might need to run some or all of the commands in these instructions with ```sudo``` or administrative privileges.\
These instructions assume that a version of PostgreSQL is already installed on the system by whatever method.

## Preparation

1. macOS: none

2. Windows: none

3. RPI:
        
    1. Make sure that the SSH client used to interact with the Raspberry Pi does not set conflicting environment variables (especially locale, as these settings are read by PostgreSQL when creating a new cluster). To do this, comment out the relevent lines in ```/etc/ssh/ssh_config```.

    2. To be sure, set system-wide locale again using ```raspi-config```.

    3. Reboot the RPI and restart the SSH connection.

## Creating a new database cluster

Do so ideally in either any of the the git-ignored folders (```websiteBot/postgresql_database/```, ```websiteBot/db/```, ```websiteBot/db_data/```) or any folder not within the websiteBot project.

1. macOS: Use the PostgreSQL tool initdb: ```initdb -D {path to db folder}```\
   *Note: The default user and owner will be the current user.*

2. Windows: Use the PostgreSQL tool initdb: ```cd``` to your "db" folder, make a subfolder "db_data" and execute: ```^"C^:^/Program^ Files^/PostgreSQL^/12^/bin^/initdb^" -D db_data```\
   *Note: The default user and owner will be the current user.*

3. RPI: Use the PostgreSQL tool pg_createcluster: ```pg_createcluster {postrgesql version} {cluster/config name} -d {path to db folder}```\
   *Note: The default user and owner will be ```postgres```.*

## Starting the database cluster server

1. macOS: Use the PostgreSQL tool pg_ctl (with optional logging): ```pg_ctl -D {path to db folder} [-l {path to logfile}] start```\
   *Note: to bind to a specific IP, use: ```pg_ctl -D {path to db folder} start [-l {path to logfile}] -o "-i -h 192.168.1.1"```.*

2. Windows: Use the PostgreSQL tool pg_ctl (with optional logging): ```^"C^:^/Program^ Files^/PostgreSQL^/12^/bin^/pg^_ctl^" -D db_data [-l {path to logfile}] start```

3. RPI:

    1. In the file ```/etc/postgresql/{postrgesql version}/{cluster/config name}/postgresql.conf``` edit the following:\
       Uncomment ```listen_addresses``` and set to localhost and IPv4 address or set just to * as an easier option.\
       Change ```data_directory``` to ```{path to db folder}```.

    2. In the file ```/etc/postgresql/{postrgesql version}/{cluster/config name}/pg_hba.conf``` edit the following:\
       Change incoming IP whitelist to IPs in own subnet, e.g. ```host  all  all  192.168.1.1/24  md5```.

    3. To start the server use:

        1. Just for manual debugging: ```pg_ctlcluster {postrgesql version} {cluster/config name} start```

        2. Actual startup: ```systemctl start postgresql@{postrgesql version}-{cluster/config name}.service```

## Creating the new database ```websitebot_db``` within the cluster

1. macOS: ```createdb websitebot_db```

2. Windows: ```^"C^:^/Program^ Files^/PostgreSQL^/12^/bin^/createdb" websitebot_db```

3. RPI: ```createdb websitebot_db```

## Connecting to the database

1. Connect to the database in Azure Data Studio. Be sure to fill out the field "active database" with ```websitebot_db```.\
   *Note: Connecting via Azure from a Unix/Windows machine to a running cluster server on an RPI via SSH is only possible if a password for the database user is set. This is because PostgreSQL can use peer authentication (i.e. "check the credentials of the currently logged in system user against the database users") only on local (i.e. localhost) connections. On top of this, the default user on the RPI when creating a new cluster is ```postgres```, so peer authentication would not work in any case. To work around this, set a password for the user ```postgres``` using ```ALTER USER postgres WITH PASSWORD '{password}';``` (do not omit the two single quote marks around the password) in the ```psql``` shell on the RPI before connecting via Azure.*

2. Via the ```psql``` shell: ```psql websitebot_db -U {username}```\
   *Note: replace ```{username}``` with the active shell login name (i.e. the user which issued the first ```initdb``` command) or - if the ```initdb``` command was issued with a ```-U {username}``` flag - the name of that user (the standard for this is ```postgres```).*\
   *Note: On RPI, instead of setting the ```-U``` flag, you usually have to switch to the "service user" of PostgreSQL with ```su postgres``` beforehand and then just issue ```psql websitebot_db```.*\
   If the login does not work (cannot ```su``` or ```-U``` to user that owns the database, no prompt for password appears, connection fails due to unsuccessful peer authentication (e.g. when using ssh on a remote system), etc.) use this long-form login pattern: ```psql "postgresql://{username}:{password}@{database server location}/{database name}"```\
   *Note: ```{database server location}``` is not the ```{cluster/config name}``` but the actual network location, e.g. ```localhost```.*

   Useful commands in the ```psql``` shell:
   | command | description |
   | ------- | ----------- |
   | ```\d  (\d+) {tablename}``` | list all columns and their data types in the given table (```+``` = more info) |
   | ```\dt (\dt+)```            | list all relations (i.e. tables) (```+``` = more info) |
   | ```\du (\du+)```            | list all users (```+``` = more info) |
   | ```\l  (\l+)```             | list all databases in the current logged in cluster (```+``` = more info) |
   | ```\t```                    | toggle display of column headers in outputs |
   | ```\pset format wrapped```  | set the columns to a fixed display width (text wraps around within column width) |
   | ```\q```                    | exit shell |

## Executing the commands in this notebook

This builds the correct structure of our ```websitebot_db``` database reliably.

1. Azure Data Studio: Just attach the notebook to the PostgreSQL connection and execute each cell one after the other. (As noted at the end of the notebook once more: Remember to "Clear Results" after finishing for a clean notebook file.)

2. ```psql``` shell: Copy and execute each command (i.e. until the next semicolon; linebreaks within one command should be copied as well) one after the other, regardless of cells.

## Stopping the database cluster server

1. macOS: Use the PostgreSQL tool pg_ctl: ```pg_ctl -D {path to db folder} stop```  

2. Windows: Use the PostgreSQL tool pg_ctl: ```^"C^:^/Program^ Files^/PostgreSQL^/12^/bin^/pg^_ctl^" -D db_data stop```\
   __TODO: is it true that there is another way on Windows if this fails?__

3. RPI:

    1. In case of manual debugging: ```pg_ctlcluster {postrgesql version} {cluster/config name} stop```

    2. In case of actual startup: ```systemctl stop postgresql@{postrgesql version}-{cluster/config name}.service```\
       *Note: To restart the server, use ```systemctl restart postgresql@{postrgesql version}-{cluster/config name}.service```, and to check its current status, use ```systemctl status postgresql@{postrgesql version}-{cluster/config name}.service```.*

## Deleting the database cluster

In case you need to start again, first make sure that the server and any services (RPI) are stopped correctly.

1. macOS: Just delete the ```{path to db folder}``` directory after checking that everything is stopped.

2. Windows: Just delete the ```{path to db folder}``` directory after checking that everything is stopped.

3. RPI: First remove the folder ```/etc/postgresql/{postrgesql version}/{cluster/config name}/``` and then delete the ```{path to db folder}``` directory, both after checking that everything is stopped.

In [ ]:
CREATE TABLE credentials (
	bot_name VARCHAR (32) NOT NULL UNIQUE,
	token VARCHAR (128) NOT NULL
);

INSERT INTO credentials (bot_name, token) VALUES ('websiteBot_bot', '***REMOVED***');
INSERT INTO credentials (bot_name, token) VALUES ('websiteBotShortTests_bot', '***REMOVED***');

In [ ]:
CREATE TABLE users (
	tg_id BIGINT NOT NULL UNIQUE,
	status SMALLINT DEFAULT 2,
	first_name VARCHAR (64) NOT NULL,
	last_name VARCHAR (64),
	username VARCHAR (64),
	apply_name VARCHAR (4096),
	apply_text VARCHAR (4096),
	apply_date DATE NOT NULL
);

INSERT INTO users (tg_id, status, first_name, last_name, username, apply_name, apply_text, apply_date) VALUES (***REMOVED***, 0, 'Niklas', 'Bogensperger', 'root', NULL, NULL, NOW());
INSERT INTO users (tg_id, status, first_name, last_name, username, apply_name, apply_text, apply_date) VALUES (***REMOVED***, 0, 'Tassilo', 'Schwarz', 'root', NULL, NULL, NOW());

In [ ]:
CREATE TABLE websites (
	ws_id SERIAL PRIMARY KEY,
	ws_name VARCHAR(64) NOT NULL UNIQUE,
	url VARCHAR (2083) NOT NULL UNIQUE,
	time_sleep SMALLINT NOT NULL,
	last_time_checked TIMESTAMP(6) WITHOUT TIME ZONE NOT NULL,
	last_time_updated TIMESTAMP(6) WITHOUT TIME ZONE NOT NULL,
	last_error_msg TEXT,
	last_error_time TIMESTAMP(6) WITHOUT TIME ZONE,
	last_hash VARCHAR(32)
);

In [ ]:
CREATE TABLE websites_content (
	ws_id INT NOT NULL,
	last_time_updated TIMESTAMP(6) WITHOUT TIME ZONE NOT NULL,
	last_hash VARCHAR(32),
	last_content TEXT,
	CONSTRAINT foreign_website FOREIGN KEY (ws_id) REFERENCES websites(ws_id) ON DELETE CASCADE,
	PRIMARY KEY (ws_id, last_time_updated)
);

In [ ]:
CREATE TABLE subscriptions (
	ws_id INTEGER NOT NULL,
	tg_id BIGINT NOT NULL,
	UNIQUE(ws_id, tg_id),
    CONSTRAINT foreign_usr FOREIGN KEY (tg_id) REFERENCES users(tg_id) ON DELETE CASCADE,
    CONSTRAINT foreign_website FOREIGN KEY (ws_id) REFERENCES websites(ws_id) ON DELETE CASCADE
);

In [ ]:
CREATE USER websitebot;
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO websitebot;
GRANT USAGE, SELECT ON ALL SEQUENCES IN SCHEMA public TO websitebot;
ALTER USER websitebot WITH PASSWORD 'webSiteBotPostGresQL';

__Remember to "Clear Results" after finishing for a clean file ;)__